In [5]:
import statsmodels.formula.api as smf
import numpy as np
import pandas as pd
from sklearn import metrics
from sklearn.model_selection import train_test_split

# import other functions
from imputer import *
from feature_eng import *
from drop import *

In [2]:
#Download the data
df = pd.read_csv('../data/train_data.zip')
df.head()

,external_id,month,year,monthly_number_of_sessions,monthly_unique_sessions,monthly_repeated_sessions,monthly_avg_length_of_session,monthly_avg_light_activity,monthly_avg_moderate_activity,monthly_avg_vigorous_activity,...,avg_wind_9_10,avg_wind_10_11,avg_wind_11_12,avg_wind_12_above,perfect_days,unacast_session_count,hpi,state_and_local_amount_per_capita,state_amount_per_capita,local_amount_per_capita
0,1804425,8,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,90.0,244.20,0.157475,0.009783,0.147692
1,1812706,2,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,27.0,258.95,0.157475,0.009783,0.147692
2,1812706,3,2019,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,4.0,27.0,258.95,0.157475,0.009783,0.147692
3,1812706,11,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,3.0,24.0,258.95,0.157475,0.009783,0.147692
4,1812706,9,2018,0,0,0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,12.0,258.95,0.157475,0.009783,0.147692


In [44]:
# drop rows missing target variable
df = drop_missing_unacast(df)
# create X and y
X = df.drop('unacast_session_count', axis=1)
y = df.loc[:, 'unacast_session_count']
# split the data
X_train, X_valid, y_train, y_valid = train_test_split(X, y, 
                                                    test_size=0.2,
                                                      random_state=2020)
# impute NaN values
result = impute_data(X_train, X_valid)
X_train = result[0]
X_valid = result[1] 
# perform feature eng
X_train = comb_cols(X_train)
X_valid = comb_cols(X_valid)
# perform dropping
X_train = drop_columns(X_train)
X_valid = drop_columns(X_valid)

# perform OHE (climate, density_class, income_class)
X_train_valid = clean_categorical(X_train, X_valid, ['income_class', 'density_class'])
X_train = X_train_valid[0]
X_valid = X_train_valid[1]

print(X_train.shape)
print(X_valid.shape)

(39592, 630)
(9898, 630)


In [68]:
valid = pd.concat([X_valid, y_valid.reset_index()], axis=1)
train = pd.concat([X_train, y_train.reset_index()], axis=1)

valid.shape

(9898, 632)

In [164]:
col = '+'.join(list(train_var.drop(columns = ['unacast_session_count']).columns))
col[:-7004]

'month+year+monthly_number_of_sessions+monthly_unique_sessions+monthly_repeated_sessions+monthly_avg_length_of_session+monthly_avg_light_activity+monthly_avg_moderate_activity+monthly_avg_vigorous_activity+monthly_under_40+monthly_temp_40_to_50+monthly_temp_50_to_60+monthly_temp_60_to_70+monthly_temp_70_to_80+monthly_temp_80_to_90+monthly_over_90+monthly_avg_length_under_40+monthly_avg_length_temp_40_to_50+monthly_avg_length_temp_50_to_60+monthly_avg_length_temp_60_to_70+monthly_avg_length_temp_70_to_80+monthly_avg_length_temp_80_to_90+monthly_avg_length_over_90+monthly_cloudy+monthly_clear+monthly_foggy+monthly_snow+monthly_avg_cloudy_length+monthly_avg_clear_length+monthly_avg_foggy_length+monthly_avg_snow_length+monthly_hour_7+monthly_hour_8+monthly_hour_9+monthly_hour_10+monthly_hour_11+monthly_hour_12+monthly_hour_13+monthly_hour_14+monthly_hour_15+monthly_hour_16+monthly_hour_17+monthly_hour_18+monthly_hour_19+monthly_hour_20+monthly_hour_21+distance_to_nearest_bus_stop+B20004e10

In [166]:
md = smf.mixedlm("unacast_session_count ~ " + col, train_var, groups=train_var["climate"])

In [162]:
def variance_threshold_selector(data, threshold=0.5):
    # https://stackoverflow.com/a/39813304/1956309
    selector = VarianceThreshold(threshold)
    selector.fit(data)
    return data[data.columns[selector.get_support(indices=True)]]

# min_variance = .9 * (1 - .9)  # You can play here with different values.
min_variance = 0.001
low_variance = variance_threshold_selector(train.drop(columns=['climate']), min_variance) 

In [153]:
train.drop(columns=['climate']).shape

(39592, 631)

In [163]:
train_var = pd.concat([low_variance, train[['climate']]], axis=1)
train_var.shape

(39592, 623)

In [167]:
mdf = md.fit(method=['bfgs', 'lbfgs', 'cg', 'bfgs'])
print(mdf.summary())
ML_MAE = metrics.mean_absolute_error(y_train, mdf.predict(X_train))
print(ML_MAE)

LinAlgError: Singular matrix

In [159]:
ML_RMSE = np.sqrt(metrics.mean_squared_error(y_train, mdf.predict(train_var)))
print(ML_RMSE)

215.09231036563318
